In [1]:
import torch
import sys
sys.path.append('../../torchgeo')
import torchgeo
import pytorch_lightning as pl

from importlib import reload

from torchgeo.trainers import EnviroatlasDataModule, EnviroatlasSegmentationTask
from torchmetrics import Accuracy, IoU, MetricCollection

%load_ext autoreload
%autoreload 2


For the highres models we can use the torchgeo dataloaders to evaluate accuracy. The QR models in constrast need to read the whole image at once to compute the normalization factors from which we compute R.

In [5]:
torchgeo_output_dir = '/home/esther/torchgeo/output'
torchgeo_data_dir = '/home/esther/torchgeo_data'

# evaulate NAIP > HR model in each state

In [4]:
ckpt_dir = f"{torchgeo_output_dir}/hp_gridsearch_pittsburgh/pittsburgh_pa-2010_1m_fcn_0.001_nll/last.ckpt"
model = EnviroatlasSegmentationTask.load_from_checkpoint(ckpt_dir)
model.freeze()
model.eval();

eval_states = ['pittsburgh_pa-2010_1m','phoenix_az-2010_1m','durham_nc-2012_1m','austin_tx-2012_1m']
# 'pittsburgh_pa-2010'
results_by_state_last_epoch = {}
for eval_state in eval_states:
    dm = EnviroatlasDataModule(
        f"{torchgeo_data_dir}/enviroatlas",
        states_str=eval_state,
        classes_keep=[1,2,3,4,6],
        batch_size=32,
        patch_size=128,
        num_workers=6,
        train_set='test', 
        val_set = 'test', 
        test_set = 'test'
    )

    trainer = pl.Trainer(gpus=[0])
    results_by_state_last_epoch[eval_state] = trainer.test(model=model, datamodule=dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


200
train sets are: ['pittsburgh_pa-2010_1m-test']
val sets are: ['pittsburgh_pa-2010_1m-test']
test sets are: ['pittsburgh_pa-2010_1m-test']


Testing: 0it [00:00, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'iou_0': 0.8937978148460388,
 'iou_1': 0.8738946914672852,
 'iou_2': 0.20835360884666443,
 'iou_3': 0.860988974571228,
 'iou_4': 0.6244174242019653,
 'test_acc_q': 0.8929929733276367,
 'test_iou_q': 0.6922904849052429,
 'test_loss': 0.0}
--------------------------------------------------------------------------------
200
train sets are: ['phoenix_az-2010_1m-test']
val sets are: ['phoenix_az-2010_1m-test']
test sets are: ['phoenix_az-2010_1m-test']


...

Testing: 0it [00:00, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'iou_0': 0.06626585870981216,
 'iou_1': 0.4306070804595947,
 'iou_2': 0.1301473081111908,
 'iou_3': 0.7963221073150635,
 'iou_4': 0.31719645857810974,
 'test_acc_q': 0.7274109721183777,
 'test_iou_q': 0.34810778498649597,
 'test_loss': 0.0}
--------------------------------------------------------------------------------
200
train sets are: ['austin_tx-2012_1m-test']
val sets are: ['austin_tx-2012_1m-test']
test sets are: ['austin_tx-2012_1m-test']


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'iou_0': 0.15695229172706604,
 'iou_1': 0.46736863255500793,
 'iou_2': 0.02773326076567173,
 'iou_3': 0.7739262580871582,
 'iou_4': 0.40982910990715027,
 'test_acc_q': 0.7176884412765503,
 'test_iou_q': 0.36716189980506897,
 'test_loss': 0.0}
--------------------------------------------------------------------------------


# evaulate NAIP+Prior > HR model in each state

In [7]:
ckpt_dir = f"{torchgeo_output_dir}/hp_gridsearch_pittsburgh_with_prior_as_input/pittsburgh_pa-2010_1m_fcn_0.001_nll/last.ckpt"
model = EnviroatlasSegmentationTask.load_from_checkpoint(ckpt_dir)
model.freeze()
model.eval();

eval_states = ['pittsburgh_pa-2010_1m','phoenix_az-2010_1m','durham_nc-2012_1m','austin_tx-2012_1m']
# 'pittsburgh_pa-2010'
results_by_state_last_epoch = {}
for eval_state in eval_states:
    dm = EnviroatlasDataModule(
        f"{torchgeo_data_dir}/enviroatlas",
        states_str=eval_state,
        classes_keep=[1,2,3,4,6],
        batch_size=32,
        patch_size=128,
        num_workers=6,
        train_set='test', 
        val_set = 'test', 
        test_set = 'test',
        include_prior_as_datalayer=True,
    )

    trainer = pl.Trainer(gpus=[0])
    results_by_state_last_epoch[eval_state] = trainer.test(model=model, datamodule=dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


200
train sets are: ['pittsburgh_pa-2010_1m-test']
val sets are: ['pittsburgh_pa-2010_1m-test']
test sets are: ['pittsburgh_pa-2010_1m-test']


Testing: 0it [00:00, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'iou_0': 0.9640726447105408,
 'iou_1': 0.8818714022636414,
 'iou_2': 0.20372101664543152,
 'iou_3': 0.8598912954330444,
 'iou_4': 0.6240530610084534,
 'test_acc_q': 0.8943305015563965,
 'test_iou_q': 0.7067219018936157,
 'test_loss': 0.0}
--------------------------------------------------------------------------------
200
train sets are: ['phoenix_az-2010_1m-test']
val sets are: ['phoenix_az-2010_1m-test']
test sets are: ['phoenix_az-2010_1m-test']


...

Testing: 0it [00:00, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'iou_0': 0.6333632469177246,
 'iou_1': 0.6201594471931458,
 'iou_2': 0.06957065314054489,
 'iou_3': 0.7966300845146179,
 'iou_4': 0.311752051115036,
 'test_acc_q': 0.7878894805908203,
 'test_iou_q': 0.48629507422447205,
 'test_loss': 0.0}
--------------------------------------------------------------------------------
200
train sets are: ['austin_tx-2012_1m-test']
val sets are: ['austin_tx-2012_1m-test']
test sets are: ['austin_tx-2012_1m-test']


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'iou_0': 0.6840725541114807,
 'iou_1': 0.6395398378372192,
 'iou_2': 0.06644535064697266,
 'iou_3': 0.7669135928153992,
 'iou_4': 0.37476789951324463,
 'test_acc_q': 0.7717442512512207,
 'test_iou_q': 0.5063478350639343,
 'test_loss': 0.0}
--------------------------------------------------------------------------------
